In [ ]:
from langchain_community.utilities.google_lens import GoogleLensAPIWrapper
from langchain_community.tools.google_lens import GoogleLensQueryRun
from langgraph.checkpoint.memory import MemorySaver
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
import os

load_dotenv()
memory = MemorySaver()

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

tool = GoogleLensQueryRun(api_wrapper=GoogleLensAPIWrapper())

In [ ]:
# Runs google lens on an image of Danny Devito
# tool.run("https://i.imgur.com/HBrB8p0.png")

In [ ]:
llm = ChatOllama(model="deepseek-v3.1:671b-cloud")

agent = create_agent(
    llm,
    tools=[tool],
    system_prompt="""
    you are a helpful assistant that can answer questions about images using google lens. 
    You have access to a tool that allows you to run google lens on an image and get the results. Use this tool to answer questions about the image.
    """,
    checkpointer=memory
)
agent

In [ ]:
config = {"configurable": {"thread_id": "12345"}}

while True:
    example_query = input("Enter your query: ")
    if example_query.lower() in ["exit", "quit"]:
        break

    response = agent.invoke(
        {"messages": [{
            "role": "user",
            "content": example_query
        }]},

        stream_mode="values", config=config
    )

    print("✅✅Agent response:")
    print(response["messages"][-1].content)